# Performing pyscenic on atlas to idenitify regulons

#### problems with pyscenic package and dependencies, therefore instead of running this in my normal environment (env) run this in environment called pyscenic (mamba activate pyscenic) as this package has the correct package versions needed

* pyscenic installed on environemnt: https://pyscenic.readthedocs.io/en/latest/installation.html#docker-podman-and-singularity-apptainer-images 
* pyscenic pipline based on: https://htmlpreview.github.io/?https://github.com/aertslab/SCENICprotocol/blob/master/notebooks/PBMC10k_SCENIC-protocol-CLI.html
* GRN inference run using arboreto method: https://pyscenic.readthedocs.io/en/latest/faq.html#i-am-having-problems-with-dask

In [1]:
#load packages I need
import os
import tools
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import pyscenic
from pyscenic.aucell import aucell

sc.logging.print_header()

scanpy==1.10.3 anndata==0.10.9 umap==0.5.7 numpy==1.23.5 scipy==1.14.1 pandas==2.2.3 scikit-learn==1.6.0 statsmodels==0.14.4 pynndescent==0.5.13


# Import h5ad

In [2]:
#set current directory 
os.chdir('/scratch/user/s4436039/scdata/Python_Integration_Sep')
os.getcwd()

'/scratch/user/s4436039/scdata/Python_Integration_Sep'

In [3]:
# read in data
data = sc.read_h5ad('NRclean_clustered2.h5ad')

In [4]:
#make two versions of the data, once containing just cDC1 and mregDC, and another containing just cDC2 and mregDC
data_DC1mreg = data[data.obs["NR_annotations_simple"].isin(["cDC1","mregDC"])]
data_DC2mreg = data[data.obs["NR_annotations_simple"].isin(["cDC2","mregDC"])]

In [2]:
os.chdir('/scratch/user/s4436039/scdata/PySCENIC')
os.getcwd()

'/scratch/user/s4436039/scdata/PySCENIC'

## make loom file with expression data:

In [10]:
import loompy as lp
import numpy as np

In [7]:
# Save the path where you want loom file to save (current directory)
f_loom_path_DC1mreg = "DC1mreg_loom.loom"

In [7]:
row_attrs = { 
    "Gene": np.array(data_DC1mreg.var.index) ,
}
col_attrs = { 
    "CellID":  np.array(data_DC1mreg.obs.index) ,
    "nGene": np.array( np.sum(data_DC1mreg.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(data_DC1mreg.X.transpose() , axis=0)).flatten() ,
}

lp.create( f_loom_path_DC1mreg, data_DC1mreg.X.transpose(), row_attrs, col_attrs )

## 1. GRN Inference - Run GRN pyscenic (20min)
* TF list (allTFs_hg38.txt) downloaded from https://resources.aertslab.org/cistarget/tf_lists/ 

In [12]:
os.chdir('/scratch/user/s4436039/scdata/PySCENIC')
os.getcwd()

'/scratch/user/s4436039/scdata/PySCENIC'

In [6]:
!arboreto_with_multiprocessing.py DC1mreg_loom.loom allTFs_hg38.txt --output adj.tsv --num_workers 1

Loaded expression matrix of 12850 cells and 1268 genes in 0.5796971321105957 seconds...
Loaded 1892 TFs...
starting grnboost2 using 1 processes...
100%|███████████████████████████████████████| 1268/1268 [20:01<00:00,  1.06it/s]
Done in 1201.6172795295715 seconds.


In [7]:
# read in the adjacencies matrix
adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')

In [8]:
adjacencies.head()

,TF,target,importance
0,IRF8,CLEC9A,43.309302
1,REL,CCR7,25.238298
2,IRF8,CCR7,25.116283
3,IRF8,CLNK,22.595391
4,IRF4,LAMP3,22.553578


# 2. Cis-Regulatory Analysis - Regulon prediction 
* output: the regulons in reg.csv - list of adjacencies between a TF and its targets stored in MOTIFS_FNAME
* ranking database downloaded from: https://resources.aertslab.org/cistarget/databases/
* motif-to-TF database downloaded from: https://resources.aertslab.org/cistarget/motif2tf/

In [10]:
# set path to ranking database (.feather)
f_db_names = "/scratch/user/s4436039/scdata/PySCENIC/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather"

In [11]:
# set path to the motif-to-TF mapping database (this database links TF binding motifs to their corresponding TFs)
f_motif_path = "/scratch/user/s4436039/scdata/PySCENIC/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl"

In [14]:
# set path to expression matrix
f_loom_path_scenic = "/scratch/user/s4436039/scdata/PySCENIC/DC1mreg_loom.loom"

### ctx step: cis-regulatory motif analysis 

In [15]:
!pyscenic ctx adj.tsv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20


2024-12-18 12:43:35,964 - pyscenic.cli.pyscenic - INFO - Creating modules.

2024-12-18 12:43:35,994 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-12-18 12:43:36,638 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-12-18 12:43:36,644 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-12-18 12:43:40,039 - pyscenic.utils - INFO - Creating modules.

2024-12-18 12:43:41,942 - pyscenic.cli.pyscenic - IN

# Regulon Activity (aucell)
* identifies the activity of these regulons at the single cell level by calculating an enrichment score

## 2. Motif enrichment and pruning the GRN (based on motif enrichment)
* to perform pruning need motif rankings and TF lists

## 3. Scoring the cells (using AUCell to determine activity)

# other

# using pyscenic to extract the regulons
* downloaded necessary databases: https://pyscenic.readthedocs.io/en/latest/installation.html
* database ranking the genome --> used Homo sapiens - hg38 - refseq_r80 - SCENIC+ databases - Gene based --> hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
* Motif to TF annotations --> 

In [3]:
# using pyscenic to extract the regulons

In [ ]:
#make expression matrix
expr_matrix = pd.DataFrame(data_DC1mreg.X, index=data_DC1mreg.obs_names, columns=data_DC1mreg.var_names)

In [ ]:
expr_matrix.to_csv('DC1mreg_expr_matrix.tsv', sep='\t', index=False)

In [ ]:
f_expr = "DC1mreg_expr_matrix.tsv" 